In [ ]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import time

# Initialize the hand detector
detector = HandDetector(detectionCon=0.7, maxHands=1)

# Capture video from the webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Define calculator layout with buttons
button_list = [['AC', '%', '/', 'x'],
               ['7', '8', '9', '-'],
               ['4', '5', '6', '+'],
               ['1', '2', '3', '='],
               ['00', '0', '.', 'B']]  # Backspace is represented as 'B'

# Button size and layout settings
button_width, button_height = 50, 50  # Reduced button size
button_offset_x = 250  # Adjusted position to move more in the middle
button_offset_y = 120  # Fixed vertical offset from the top

user_input = ""
result = ""

# To introduce a delay between button presses
last_button_press_time = 0
button_press_delay = 1  # 1-second delay between button presses

# Function to draw buttons with an attractive look
def draw_buttons(img, highlighted_button=None):
    img_overlay = img.copy()
    for i in range(len(button_list)):
        for j in range(len(button_list[i])):
            x = button_offset_x + j * button_width
            y = button_offset_y + i * button_height
            
            # Highlight button if it is pressed
            if highlighted_button == button_list[i][j]:
                color = (0, 150, 0)  # Green highlight
            else:
                color = (50, 50, 50)  # Default button color
            
            # Draw rounded rectangle for the button
            cv2.rectangle(img_overlay, (x, y), (x + button_width, y + button_height), color, -1, cv2.LINE_AA)
            # Add shadows (optional)
            cv2.rectangle(img_overlay, (x + 3, y + 3), (x + button_width + 2, y + button_height + 2), (30, 30, 30), -1)  # Shadow

            # Add button text with larger font size
            cv2.putText(img_overlay, button_list[i][j], (x + 10, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw a border around the calculator
    cv2.rectangle(img_overlay, (button_offset_x - 5, button_offset_y - 5), 
                  (button_offset_x + 4 * button_width + 5, button_offset_y + 5 * button_height + 5), 
                  (200, 200, 250), 2)  # Light border color

    return img_overlay

# Function to check if button is pressed
def button_pressed(finger_pos):
    for i in range(len(button_list)):
        for j in range(len(button_list[i])):
            x = button_offset_x + j * button_width
            y = button_offset_y + i * button_height
            if x < finger_pos[0] < x + button_width and y < finger_pos[1] < y + button_height:
                return button_list[i][j]
    return None

# Create a named window to set fullscreen
cv2.namedWindow("Virtual Transparent Calculator with Gestures", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Virtual Transparent Calculator with Gestures", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Main loop
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # Flip the image for better user experience

    # Draw transparent buttons with highlighting
    highlighted_button = None
    img_overlay = draw_buttons(img, highlighted_button)

    # Detect hands and find landmarks
    hands, img = detector.findHands(img, flipType=False)

    if hands:
        lmList = hands[0]['lmList']  # Get the landmark list for the first hand
        if len(lmList) >= 12:
            # Get index finger tip coordinates
            index_finger = (lmList[8][0], lmList[8][1])
            thumb_finger = (lmList[4][0], lmList[4][1])  # Thumb tip coordinates

            # Check if a button is pressed using index finger
            button = button_pressed(index_finger)

            # Calculate the distance between thumb and index finger to confirm the press
            distance, _, img = detector.findDistance(index_finger, thumb_finger, img)

            # Highlight button if pressed
            if button and distance < 40:
                highlighted_button = button
            
            # Add a time-based delay to slow down reaction speed
            current_time = time.time()

            if distance < 40 and button and current_time - last_button_press_time > button_press_delay:
                # Handle button press logic
                if button == '=':
                    try:
                        # Replace 'x' with '*' for multiplication to evaluate correctly
                        expression = user_input.replace('x', '*')
                        result = str(eval(expression))  # Calculate result
                        user_input = result  # Display result as new input
                    except:
                        result = "Error"
                        user_input = ""
                elif button == 'AC':  # Handle All Clear
                    user_input = ""
                    result = ""
                elif button == 'B':  # Handle backspace
                    user_input = user_input[:-1]  # Remove last character
                elif button == '00':  # Handle double zeroes
                    user_input += "00"
                else:
                    if result != "Error":  # Prevent input after error
                        user_input += button

                last_button_press_time = current_time  # Update the time of the last button press

    # Display user input and result
    cv2.rectangle(img_overlay, (button_offset_x - 10, 20), (button_offset_x + 4 * button_width + 10, 120), (230, 230, 250), -1)  # Light background
    cv2.putText(img_overlay, user_input, (button_offset_x + 10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)

    # Show the result if there's one
    if result:
        cv2.putText(img_overlay, result, (button_offset_x + 10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 150, 0), 2)

    # Blend the transparent overlay with the original image
    final_img = cv2.addWeighted(img, 0.7, img_overlay, 0.8, 0)  # Increased transparency

    # Display the final image
    cv2.imshow("Virtual Transparent Calculator with Gestures", final_img)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


C:\Users\shiva\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
